# Cloud Assignment 1

### 1. The top 10 posts by score

In [2]:
#importing libraries
from pyhive import hive
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

#creating connection instance to set up connectiion with hive
conn = hive.Connection(host="127.0.0.1", port=10000, username="hdoop")

dataframe = pd.read_sql("SELECT OWNERDISPLAYNAME,TITLE,SCORE,BODY FROM STACKEXDATA ORDER BY SCORE DESC LIMIT 10", conn)
display(dataframe)

,ownerdisplayname,title,score,body
0,GManNickG,Why is processing a sorted array faster than p...,25933,Here is a piece of C code that shows some very...
1,Hamza Yerlikaya,How do I undo the most recent local commits in...,23348,I accidentally committed the wrong files to Gi...
2,Matthew Rankin,How do I delete a Git branch locally and remot...,18514,I want to delete a branch both locally and rem...
3,pupeno,What is the difference between 'git pull' and ...,12834,What are the differences between git pull and ...
4,Alex. S.,"""What does the """"yield"""" keyword do?""",11551,What is the use of the yield keyword in Python...
5,Oli,What is the correct JSON content type?,10921,Ive been messing around with JSON for some tim...
6,paxos1977,How do I undo 'git add' before commit?,10079,I mistakenly added files to Git using the comm...
7,Walker,How can I remove a specific item from an array?,9931,I have an array of numbers and Im using the pu...
8,Forrest,How do I rename a local Git branch?,9792,I dont want to rename a remote branch as descr...
9,GManNickG,"""What is the """"-->"""" operator in C/C++?""",9560,After reading Hidden Features and Dark Corners...


### 2. The top 10 users by post score

In [3]:
df2 = pd.read_sql("SELECT OWNERUSERID,OWNERDISPLAYNAME,SUM(SCORE) AS TOTAL_SCORE FROM STACKEXDATA GROUP BY OWNERUSERID,OWNERDISPLAYNAME ORDER BY TOTAL_SCORE DESC LIMIT 10", conn)
display(df2)

,owneruserid,ownerdisplayname,total_score
0,87234,GManNickG,37672
1,4883,readonly,28817
2,9951,e-satis,26878
3,6068,pupeno,25944
4,89904,Hamza Yerlikaya,24024
5,51816,Joan Venge,23763
6,49153,Ali,20203
7,179736,TIMEX,19603
8,95592,Matthew Rankin,19479
9,63051,flybywire,19362


### 3.The number of distinct users, who used the word “cloud” in one of their posts

In [4]:
df3 = pd.read_sql('SELECT COUNT(DISTINCT(OwnerUserId)) AS NO_OF_USERS FROM STACKEXDATA WHERE TITLE LIKE "% cloud %" OR BODY LIKE "% cloud %"', conn)
display(df3)

,no_of_users
0,163


#### Making a data frame 'df4' of the titles and body of the top 10 users.  

In [15]:
df4 = pd.read_sql('''
SELECT OWNERUSERID, OWNERDISPLAYNAME,TITLE,BODY FROM STACKEXDATA WHERE OWNERUSERID IN (87234,4883,9951,6068,89904,51816,49153,179736,95592,63051)
''', conn)
display(df4)


,owneruserid,ownerdisplayname,title,body
0,89904,Hamza Yerlikaya,How do I undo the most recent local commits in...,I accidentally committed the wrong files to Gi...
1,95592,Matthew Rankin,How do I delete a Git branch locally and remot...,I want to delete a branch both locally and rem...
2,4883,readonly,How can you find out which process is listenin...,How can you find out which process is listenin...
3,49153,Ali,How to replace all occurrences of a string in ...,I have this string in my JavaScript code quotT...
4,6068,pupeno,How do you get a timestamp in JavaScript?,Something similar to Unixs timestamp that is a...
...,...,...,...,...
420,51816,Joan Venge,Clamping floating numbers in Python?,Is there a builtin function for this in Python...
421,9951,e-satis,Is there any Python-like interactive console f...,I spent a lot of time programming in Java rece...
422,51816,Joan Venge,Checking delegates for null,I was reading the Essential C book and am w...
423,179736,TIMEX,How can I remove all nil elements in a Swift a...,Basic way doesnt work for index in lt listco...


In [16]:
df4['title_body'] = df4[['title', 'body']].agg(' '.join, axis=1)
del df4['title']
del df4['body']
display(df4)

,owneruserid,ownerdisplayname,title_body
0,89904,Hamza Yerlikaya,How do I undo the most recent local commits in...
1,95592,Matthew Rankin,How do I delete a Git branch locally and remot...
2,4883,readonly,How can you find out which process is listenin...
3,49153,Ali,How to replace all occurrences of a string in ...
4,6068,pupeno,How do you get a timestamp in JavaScript? Some...
...,...,...,...
420,51816,Joan Venge,Clamping floating numbers in Python? Is there ...
421,9951,e-satis,Is there any Python-like interactive console f...
422,51816,Joan Venge,Checking delegates for null I was reading the ...
423,179736,TIMEX,How can I remove all nil elements in a Swift a...


In [18]:
vectorizer = TfidfVectorizer(stop_words='english', lowercase=True)
#extracting the top 10 users in a list
userlist = list(df4["ownerdisplayname"].unique())

#iterating through the list of users
for user in userlist:
    
    #storing the posts per top 10 users in a new dataframe
    user_posts_df=df4[df4["ownerdisplayname"] == user]
    
    response = vectorizer.fit_transform(user_posts_df["title_body"])
    tf_idf_df= pd.DataFrame(response.toarray(),columns=vectorizer.get_feature_names())
    
    #taking the sum of tf-idf's of each term
    sums = tf_idf_df.sum(axis=0)
    
    #finding the top 10 terms from the list of terms based on sums
    topTen= sums.nlargest(10)
    
    #making a list of the top 10 terms
    topTenWordsList = list(topTen.index)
    
    #storing the tf-idfs in a new dataframe
    tf_idf_topTen=tf_idf_df[topTenWordsList]
    
    #displaying the tf-idf's of top 10 terms of top 10 users
    display("TF-IDF of the top 10 terms for the user : "+user)
    display(tf_idf_topTen)   

'TF-IDF of the top 10 terms for the user : Hamza Yerlikaya'

,file,timer,new,java,setpreferredwidth,tablegetcolumnmodelgetcolumn,files,application,jar,array
0,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.085641,0.000000,0.000000,0.000000
1,0.000000,0.0000,0.000000,0.040077,0.606105,0.606105,0.000000,0.000000,0.000000,0.000000
2,0.351468,0.0000,0.000000,0.000000,0.000000,0.000000,0.312480,0.000000,0.000000,0.000000
3,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.463315,0.0000,0.000000,0.000000,0.000000,0.000000,0.102980,0.264787,0.264787,0.000000
5,0.000000,0.0000,0.000000,0.313903,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.6912,0.235033,0.091408,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.077501,0.0000,0.442922,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.097851,0.251599,0.251599,0.000000
9,0.000000,0.0000,0.000000,0.163094,0.000000,0.000000,0.000000,0.000000,0.000000,0.493305


'TF-IDF of the top 10 terms for the user : Matthew Rankin'

,install,python,pip,branch,installed,navigation,flask,version,started,psycopg
0,0.000000,0.000000,0.000000,0.697465,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.551288,0.000000,0.000000,0.000000,0.000000
2,0.464973,0.290143,0.286137,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.339622,0.000000,0.135849,0.000000,0.157430,0.000000,0.000000,0.000000,0.000000,0.516576
4,0.000000,0.000000,0.000000,0.000000,0.436263,0.000000,0.520551,0.520551,0.000000,0.000000
5,0.427251,0.000000,0.284834,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.432970,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.516622,0.000000


'TF-IDF of the top 10 terms for the user : readonly'

...

,python,want,string,user,dictionary,file,return,use,just,list
0,0.296315,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
1,0.168368,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
2,0.000000,0.00000,0.000000,0.000000,0.000000,0.214901,0.000000,0.086508,0.0,0.0
3,0.000000,0.00000,0.188953,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
4,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
110,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.283947,0.000000,0.0,0.0
111,0.061815,0.00000,0.000000,0.000000,0.086926,0.000000,0.000000,0.000000,0.0,0.0
112,0.000000,0.04225,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
113,0.000000,0.00000,0.000000,0.084033,0.000000,0.000000,0.092008,0.000000,0.0,0.0


'TF-IDF of the top 10 terms for the user : flybywire'

,file,python,want,vs,standard,bash,command,java,instance,output
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.112601,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.113048,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.243946,0.000000,0.000000,0.000000,0.547364,0.181656,0.000000,0.000000,0.000000,0.320451
5,0.166264,0.000000,0.155217,0.000000,0.466327,0.412698,0.000000,0.000000,0.000000,0.218406
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.588666,0.000000,0.000000
7,0.112736,0.127016,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.537229,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.122889,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


'TF-IDF of the top 10 terms for the user : GManNickG'

,idiom,copy,array,int,arraysize,data,operator,does,mentioned,places
0,0.000000,0.000000,0.260409,0.173606,0.304361,0.304361,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.149462,0.000000,0.000000,0.149462,0.000000,0.000000,0.000000
2,0.480997,0.360748,0.091453,0.000000,0.000000,0.000000,0.091453,0.240499,0.240499,0.240499
